In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [ ]:
Queen_train = np.load('Queen_train.npy')
Queen_val = np.load('Queen_val.npy')
Queen_test = np.load('Queen_test.npy')
Queen_train.shape,Queen_val.shape,Queen_test.shape

In [ ]:
Queen = np.concatenate((Queen_train,Queen_val,Queen_test))
Queen.shape

In [ ]:
NonQueen_train = np.load('NonQueen_train.npy')
NonQueen_val = np.load('NonQueen_val.npy')
NonQueen_test = np.load('Nonqueen_test.npy')
NonQueen_train.shape,NonQueen_val.shape,NonQueen_test.shape

In [ ]:
NonQueen = np.concatenate((NonQueen_train,NonQueen_val,NonQueen_test))
NonQueen.shape

In [10]:
Label_Queen = np.ones(len(Queen)) * 1  
Label_NonQueen = np.ones(len(NonQueen)) * 0  

In [11]:
data = np.concatenate((Queen,NonQueen), axis=0)
labels = np.concatenate((Label_Queen,Label_NonQueen), axis=0)

df = pd.DataFrame({'data': list(data), 'labels': labels})

In [12]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['labels'])
train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42, stratify=train_df['labels'])

x_train = np.array(train_df['data'].tolist())
y_train = np.array(train_df['labels'].tolist())

x_val = np.array(val_df['data'].tolist())
y_val = np.array(val_df['labels'].tolist())

x_test = np.array(test_df['data'].tolist())
y_test = np.array(test_df['labels'].tolist())

In [13]:
class BeeDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [14]:
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy array to PIL Image
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [15]:
train_dataset = BeeDataset(x_train, y_train, transform=transform)
val_dataset = BeeDataset(x_val, y_val, transform=transform)
test_dataset = BeeDataset(x_test, y_test, transform=transform)

In [16]:
batch_size = 64

In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [19]:
seed = 42
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [20]:
from timm import create_model

model = create_model('vit_tiny_patch16_224', pretrained=False, num_classes=2)
model.patch_embed.proj = nn.Conv2d(1, model.patch_embed.proj.out_channels, 
                                   kernel_size=model.patch_embed.proj.kernel_size, 
                                   stride=model.patch_embed.proj.stride, 
                                   padding=model.patch_embed.proj.padding)

# load model weights
# weight_path = 'path/to/weight.pth'
# state_dict = torch.load(weight_path)
# model.load_state_dict(state_dict)

model = model.to(device)
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training and validation loop
num_epochs = 50
train_accs = []
val_accs = []
train_losses = []
val_losses = []
best_val_accuracy = 0
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_corrects = 0
    for train_images, train_labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
        train_images = train_images.to(device)
        train_labels = train_labels.long().to(device)
        optimizer.zero_grad()
        outputs = model(train_images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * train_images.size(0)
        train_corrects += torch.sum(preds == train_labels.data)
    train_loss /= len(train_dataloader.dataset)
    train_acc = train_corrects.double() / len(train_dataloader.dataset)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    # Validation
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1} Validation"):
            val_images = val_images.to(device)
            val_labels = val_labels.long().to(device)
            
            val_outputs = model(val_images)
            _, preds = torch.max(val_outputs, 1)
            loss = criterion(val_outputs, val_labels)
            
            val_loss += loss.item() * val_images.size(0)
            val_corrects += torch.sum(preds == val_labels.data)
    val_loss /= len(val_dataloader.dataset)
    val_acc = val_corrects.double() / len(val_dataloader.dataset)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc:{val_acc:.4f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'best_model_{learning_rate}.pth')